# Main Imports

In [ ]:
from cnn_super_resolution import model
from model_saver_manager import model_saver
from prepare_dataset.dataset_manager import DataSetManager
import numpy as np
import cv2

# Model Initialization

In [ ]:
model_instance = model.SRModel()
parameters = model_instance.get_parameter_tensors()

modelsave = model_saver.ModelSaver(
    'exp1_', parameters,
    save_file_path="/media/sreramk/storage-main/elementary_frame/checkpoints/")

model_instance.set_model_saver_inst(modelsave)

## Prepare dataset

In [ ]:
model_instance.prepare_train_test_dataset(
    ['/media/sreramk/storage-main/elementary_frame/dataset/DIV2K_train_HR/'],
    ['/media/sreramk/storage-main/elementary_frame/dataset/DIV2K_valid_HR/'],
    num_of_training_ds_to_load=160, num_of_testing_ds_to_load=40,
    train_batch_size=10, testing_dimension=(500, 500)
    )

## Set the train/test loss function(s)

In [ ]:
#model_instance.set_rms_loss()
#model_instance.set_rms_triplet_loss()
model_instance.set_psnr_loss()
#model_instance.set_psnr_triplet_loss()

## Obtain the image to test with

In [ ]:
#model_instance.run_test()
#print("active loss: " + model_instance.get_active_loss())
#img = np.zeros((500,500,3))
#img[:, :] = (0.0, 0.0, 0.0)

img = model_instance.fetch_image('/media/sreramk/storage-main/elementary_frame/dataset/DIV2K_valid_HR/0850.png', # '/media/sreramk/storage-main/elementary_frame/dataset/t2.png', #'/media/sreramk/storage-main/elementary_frame/dataset/DIV2K_valid_HR/0891.png',
                                 with_batch_column=False)

#down_sampled = cv2.resize(img, dsize=(int(len(img[0]) / 4),
#                                               int(len(img) / 4)), interpolation=cv2.INTER_NEAREST)

#down_sampled = cv2.resize(down_sampled, dsize=(len(img[0]), len(img)),
#                                  interpolation=cv2.INTER_NEAREST)
#img = cv2.GaussianBlur(down_sampled, (11, 11), 0)
##img = 1-img
#img = np.flip(img, axis=0)
model_instance.display_image(img, figure_size=(50, 50), invert_colors=True)
#print (1-img)

# Run the training

In [ ]:
modelsave.checkpoint_model_arguments(skip_duration=100)
model_instance.set_learning_rate(0.01)
model_instance.set_momentum(0.01)
model_instance.set_optimization_level(1)
for i in range(1000):
    model_instance.run_train(num_of_epochs=10, reinitialize_train_batch=True, rebase_checkpoints=True)

# Explore the result

In [ ]:
_, __, img = DataSetManager.random_crop_img(img, 200, 200)
img = model_instance.zoom_image(img, 4, 4)

In [ ]:
for i in range(2):
    # cv2.imshow("im1", img)
    model_instance.display_image(img,  figure_size=(70, 70), invert_colors=False)
    result = model_instance.execute_model(input_image=img, return_with_batch_column=False)
    # print(result)
    model_instance.display_image(result, figure_size=(70, 70))
    # cv2.imshow("img2", result)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    img = result

img = model_instance.fetch_image(
    "https://cdn.insidetheperimeter.ca/wp-content/uploads/2015/11/Albert_einstein_by_zuzahin-d5pcbug-WikiCommons-768x706.jpg")
size_x, size_y = model_instance.get_image_dimensions(img)

%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import numpy

In [ ]:
x= numpy.array([1])

In [ ]:
x = x*3

In [ ]:
x

# Rough work

In [ ]:
import cv2

In [ ]:
_, __, img2 = DataSetManager.random_crop_img(img, 100, 100)

down_sampled = cv2.resize(img2, dsize=(int(len(img2[0]) / 4),
                                           int(len(img2) / 4)), interpolation=cv2.INTER_NEAREST)

down_sampled = cv2.resize(down_sampled, dsize=(len(img2[0]), len(img2)),
                          interpolation=cv2.INTER_NEAREST)

blur = cv2.GaussianBlur(down_sampled,(11,11),0)
smooth  = blur
#blur = cv2.GaussianBlur(blur,(5,5),0)
#smooth = cv2.addWeighted(blur,1.5,img,-0.5,0)
#_, __, smooth = DataSetManager.random_crop_img(smooth, 250, 250)

In [ ]:
model_instance.display_image(smooth,  figure_size=(70, 70))
model_instance.display_image(img2,  figure_size=(70, 70))

In [ ]:

_, __, img2 = DataSetManager.random_crop_img(img, 250, 250)
model_instance.display_image(img2, figure_size=(70, 70))

In [ ]:
dir(model_instance)

In [ ]:
import inspect
import numpy as np
inspect.signature(model_instance.get_parameter_tensors)

In [ ]:
x = model_instance.get_parameter_tensors()

In [ ]:
sess = model_instance.get_session()

In [ ]:
y = sess.run(x)

In [ ]:
y

In [ ]:
len (y)

In [ ]:
len(y[5])

In [ ]:
y1 = y[0][0:10, 0:10, 2,0]

In [ ]:
y1

In [ ]:
def display_filter(fimg):
    max_val = np.max(fimg)
    
    
    #new_shape = [3]
    #new_shape.extend(fimg.shape)
    
    new_shape = list(fimg.shape)
    new_shape.extend([3])
    fimg_color = np.zeros(shape=new_shape)
    
    fimg_color[:,:,0] = np.where(fimg>0, fimg, 0)
    fimg_color[:,:,2] = np.where(fimg<0, np.abs(fimg), 0)
    fimg_color[:,:,1] = np.where(fimg==0, max_val, 0)
    #print(fimg_color.shape)

    #print (fimg_color)

    model_instance.display_image(fimg_color, figure_size=(70, 70), invert_colors=True)

In [ ]:
np.sum(y[0][0:10, 0:10, 2,2])

In [ ]:

fimg = y[0][0:10, 0:10, 2,0]#*np.abs(y[0][0:10, 0:10, 2,0])
for i in range(1, 10):
    temp = y[0][0:10, 0:10, 2,i] *np.abs(y[0][0:10, 0:10, 2,i])
    fimg += y[0][0:10, 0:10, 2,i]#*np.abs(y[0][0:10, 0:10, 2,i])
    display_filter(temp)
#fimg = fimg/10
#fimg = fimg*np.abs(fimg)
display_filter(fimg)

print ("Random:")

fimg = np.random.uniform(-1,1, fimg.shape)
display_filter(fimg)
for i in range(1, 10):
    fimg += np.random.uniform(-1,1, fimg.shape)
    
#print (fimg)
fimg = fimg
#fimg = fimg*np.abs(fimg)
display_filter(fimg)


In [ ]:
x = None
for i in range(32):
    for j in range (3):
        if x is None:
            x = y[5][0:10, 0:10, 0,0]
        else:
            x += y[5][0:10, 0:10, i,j]
x = x/(32*3)

In [ ]:
import numpy
numpy.max(x)

In [ ]:
np.sum(y[0][0:10, 0:10, 2,0])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,1])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,2])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,3])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,4])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,5])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,6])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,7])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,8])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,9])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,10])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,11])

In [ ]:
np.sum(y[0][0:10, 0:10, 2,12])